# Recommender system

## 1. Prerequiste

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow
import tensorflow as tf
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp

%load_ext autoreload
%autoreload 2

In [17]:
from helpers import*
from preprocessing import*
from RMSE import*
from factorization_SGD import*
from ImplementingBaseline import*

In [3]:
DATA_TRAIN_PATH = "data/data_train.csv"
data = load_data(DATA_TRAIN_PATH)

number of users : 10000, number of items: 1000


## 2. Split the data

For reproductibility, remove seed from split_data

In [6]:
valid_ratings, train, test = split_data(data, min_num_ratings=0, p_test=0.1)

Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1065327
Total number of nonzero elements in test data:111625


## 3. Implementing Baseline

In [10]:
baseline_global_mean(train, test)

test RMSE of baseline using the global mean: [[1.12152228]].


In [13]:
baseline_user_mean(train, test)

test RMSE of the baseline using the user mean: [[1.03317038]].


In [14]:
baseline_item_mean(train, test)

test RMSE of the baseline using the item mean: [[1.09633198]].


## 4. Matrix Factorization using SGD

In [7]:
num_features = 20   # K in the lecture notes
lambda_user = 0.05
lambda_item = 0.2

user_features, item_features = matrix_factorization_SGD(train, num_features, lambda_user, lambda_item) 

In [8]:
nz_row, nz_col = test.nonzero()
nz_test = list(zip(nz_row, nz_col))
rmse = compute_error(test, user_features, item_features, nz_test)
print("RMSE on test data: {}.".format(rmse))

RMSE on test data: 1.0082670439014565.


In [9]:
DATA_FINAL_PATH = "data/sampleSubmission.csv"
samples = load_data(DATA_FINAL_PATH)
_, _, samples = condition_min_num_ratings(samples, min_num_ratings=0)

number of users : 10000, number of items: 1000


In [10]:
nz_row, nz_col = samples.nonzero()
nz_samples = list(zip(nz_row, nz_col))
rmse = compute_error(samples, user_features, item_features, nz_samples)
print("RMSE on test data: {}.".format(rmse))

RMSE on test data: 0.9373349757846028.


In [51]:
prediction = user_features.T.dot(item_features)
data = read_txt(DATA_FINAL_PATH)[1:]
data = [deal_line(line) for line in data]
n = len(data)
submission = np.zeros((n,1))
for i in range(n):
    submission[i] = np.clip(round(prediction[data[i][1]-1][data[i][0]-1]), 1, 5)

In [52]:
from helpers import*
DATA_SUBMISSION = "data/submission_baseline.csv"
create_csv(data, submission, DATA_SUBMISSION)

Baseline dot product method without min_num_ratings - AICrowd : 1.046 

## 4. Matrix Factorization using ALS